## Final Project
**By Tamila Kats**

[Tableau Dashboard](https://public.tableau.com/views/Project_Product_Range/Dashboard2?:language=en-US&:display_count=n&:origin=viz_share_link)

[Presentation](https://drive.google.com/file/d/1UWvtQnUqJMVUMtdg0BaHmX93URG4X7ID/view?usp=sharing)

# Product Range Analysis

We have data on sales of online store contains order number, item identifier and name, purchased quantity, price per item, order date and customer id. We are going to analyze this information to define categories of products by price, which of them often bought in big quantity, how many orders customers usually make, what groups we can form by the average check, whether demand is change from season to season, etc. We will provide recommendations for increasing sales figures.

### Table of Content:

* [ General Overview and Preprocessing](#general)
* [Defining categories](#categories)
  * [Study distribution of prices](#price)
  * [Parsing of names of products](#names)
  * [Defining proportions of regular and casual customers](#loyal)
  * [Categorizing customers by average check](#avg)
* [Identifying the seasonality of demand](#season)
  * [Checking for general increasing of demand by season](#season_gen)
  * [Checking for special interest to certain categories of products by season](#season_cat)
* [Analysis of carts](#carts)
  * [Define products that often bought together](#together)
  * [Define which products often bought in big amount](#bigamount)
* [Analysis of canceled orders](#canceled)
* [Testing statistical hypotheses](#hypotheses)
  * [If there is a statistically significant difference between revenue from small products that were bought in big amounts and big expensive products that bought one at a time?](#first)
  * [If there is a statistically significant difference between revenue in summer and winter?](#second)
* [Overall conclusion and recommendations](#overall)

## General Overview and Preprocessing <a class="anchor" id="general"></a>

In [ ]:
#importing all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
import warnings 
warnings.filterwarnings("ignore")


In [ ]:
# Libraries for text preprocessing
import re
import nltk
#nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
#! pip install mlxtend 
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
try:
    ecommerce_data = pd.read_csv('/Users/tamilaz/Downloads/ecommerce_dataset_us.csv', sep='\t')
except:
    ecommerce_data = pd.read_csv('/datasets/ecommerce_dataset_us.csv', sep='\t')


In [ ]:
ecommerce_data.head()

In [ ]:
ecommerce_data.tail()

In [ ]:
ecommerce_data.info()

In [ ]:
#changing names of columns to more convenient
ecommerce_data.columns=['invoice_no', 'stock_code', 'description', 
                        'quantity', 'invoice_date', 'unit_price', 'customer_id']

In [ ]:
#checking for null values
ecommerce_data.isnull().sum()

In [ ]:
#checking for percentage of null values in column 'description'
print('Percent of null values in column "description": {:.2f}%'.format(
    ecommerce_data['description'].isnull().sum()*100 / len(ecommerce_data)
))


We will remove these null values, because without description we couldn't analyze these products and 0.27% is not significant piece of the dataset.

In [ ]:
#dropping rows with null values
ecommerce_data.dropna(subset=['description'], inplace=True)
ecommerce_data.shape[0]

In [ ]:
#checking for percentage of null values in column 'customer_id'
print('Percent of null values in column "customer_id": {:.2f}%'.format(
    ecommerce_data['customer_id'].isnull().sum()*100 / len(ecommerce_data)
))

There is a significant part of a dataset with null values in column 'customer_id'. We can't remove it and the goal of our analysis is products and not customers so we will leave all this rows, but fill null with value 'unknown'.

In [ ]:
#filling NA with 'unknown'
ecommerce_data['customer_id'] = ecommerce_data['customer_id'].fillna('unknown')


In [ ]:
#checking for duplicates
print('Number of duplicates is {}.'.format(ecommerce_data.duplicated().sum()))

In [ ]:
#checking for percentage of duplicates for all data
print('Share of duplicates in the dataset is {:.2f}%.'.format(
    ecommerce_data.duplicated().sum()*100 / len(ecommerce_data)
))


We will remove the duplicates because it's share is not significant and there are most likely mistakes of order registration.

In [ ]:
#dropping the duplicates 
ecommerce_data = ecommerce_data.drop_duplicates().reset_index(drop=True)

In [ ]:
#changing datatypes to correct ones
ecommerce_data['quantity'] = pd.to_numeric(ecommerce_data['quantity'], downcast='unsigned')
ecommerce_data['invoice_date'] = ecommerce_data['invoice_date'].astype('datetime64')
ecommerce_data['unit_price'] = pd.to_numeric(ecommerce_data['unit_price'], downcast='float')

ecommerce_data.dtypes


In [ ]:
#extracting the date from invoice_date for more convenient further analysis
ecommerce_data['invoice_date'] = ecommerce_data['invoice_date'].dt.date
ecommerce_data.head()

In [ ]:
ecommerce_data.dtypes

In [ ]:
ecommerce_data['invoice_date'] = pd.to_datetime(ecommerce_data['invoice_date'], format='%Y-%m-%d')
ecommerce_data.dtypes

We want to check values of column 'stock_code' on special/strange values(non-numeric):

In [ ]:
#searching for all non-numeric values of stock code
codes = ecommerce_data[ecommerce_data['stock_code'].str.contains('^[a-zA-Z]+') == True]
codes.shape

In [ ]:
#checking what kinds of special codes there are 
print('Non-numeric/irrelevant values in column "stock_code": {}'.format(
    codes['stock_code'].unique()
))

In [ ]:
#checking the percentage  of rows with special/strange codes
print('Share of rows with irrelevant values in column "stock_code": {:.2f}%'.format(
    len(codes) * 100/ len(ecommerce_data)
))


We'll remove rows with all special stock codes because they are not relevant for our analysis:

In [ ]:
#removing irrelevant values
ecommerce_data = ecommerce_data.drop(codes.index).reset_index(drop=True)
ecommerce_data.shape[0]

We want to check the number of canceled orders(values in column "invoice_no" starts with "C"):

In [ ]:
#calculating the number of cancelations
canceled = ecommerce_data.loc[ecommerce_data['invoice_no'].str.startswith('C'), 'invoice_no']
print('Number of canceled orders is {}.'.format(canceled.count()))


We will split our dataset on two: one for all orders and second only for canceled. We will analyze them separately. Our purpose is to analyze what products are popular and profitable so we want to analyze all of them even if in the end the order was canceled for some reason.

In [ ]:
#creating a new df with orders without cancelations
ecommerce_data_orders= ecommerce_data.drop(ecommerce_data[ecommerce_data['invoice_no'].str.startswith('C')].index).reset_index(drop=True)
ecommerce_data_orders.shape[0]


In [ ]:
#creating a df with canceled orders
ecommerce_data_canceled= ecommerce_data[ecommerce_data['invoice_no'].str.startswith('C')]
ecommerce_data_canceled.head()


## Conclusion

We got a dataset with 541909 entries that contain information about orders made in the store: order number and date, identifier and name of the product, purchased quantity, price per unit, customer id. 

We changed all column names to more convenient and changed datatypes to correct ones. We found null values in column ‘description’ and removed them because its share was insignificant - 0.27% and it is impossible to analyze products without names. Null values in column ‘customer_id’ were filled with value ‘unknown’ because we want to have as many as possible ordered products for analysis and this is a significant part of the dataset - 24.72%. We removed all duplicates,  there were 0.97%. 

We also cleaned the data from irrelevant values in column ‘stock_code’, there were non-numeric special and strange codes that were not needed for our analysis.  Finally, we split our dataset into two - one with all orders and the second only with canceled to analyze them separately afterward.

## Categorizing of products and customers <a class="anchor" id="categories"></a>

At this stage, we are going to define categories of products by their names and prices, and of customers by their regularity and average check.

At first we will check general numbers: total amount of orders, unique products and customers: 

In [ ]:
#calculating the total number of orders(with cancellations)
print('Total number of orders: {}.'.format(
    ecommerce_data_orders['invoice_no'].nunique()
))

In [ ]:
#calculating the number of disctinct products that were bought
print('Number of unique product names: {}.'.format(
    ecommerce_data_orders['stock_code'].nunique()
))

In [ ]:
#calculating the number of customers
print('Number of customers: {}.'.format(
    ecommerce_data_orders['customer_id'].nunique()
))

Now we will define categories of products by price: <a class="anchor" id="price"></a>

In [ ]:
ecommerce_data_orders['unit_price'].describe()

Min price is 0, it is most likely a mistake, anomaly that we should check.

In [ ]:
#checking products with price 0
ecommerce_data_orders[ecommerce_data_orders['unit_price']== 0].sample(10)


In [ ]:
#calculating the percentage of units with price 0
print('The percentage of values 0 in column "unit_price" - {:.1f}%.'.format(
    len(ecommerce_data_orders[ecommerce_data_orders['unit_price']== 0]) *100 / len(ecommerce_data_orders)
))


There are strange descriptions and quantities, many unknown customers and it's only 0.2% of the dataset so we will remove these rows:

In [ ]:
#removing rows with irrelevant data
ecommerce_data_orders = ecommerce_data_orders.drop(ecommerce_data_orders[ecommerce_data_orders['unit_price']== 0].index).reset_index(drop=True)
ecommerce_data_orders.shape[0]

In [ ]:
ecommerce_data_orders['unit_price'].describe()

In [ ]:
#plot a histogram of distribution of prices
fig=px.histogram(ecommerce_data_orders, x='unit_price', title='Distribution of prices')
fig.update_layout(xaxis_title='Price', yaxis_title='Count')
fig.show()

We want to categorize bought products by price for cheap and expensive. We see that most of a products are very cheap around 0 and there are some outliers with the biggest value 649.5. First of all we will calculate percentiles to try to make a categories:

In [ ]:
#calculating 0.25 quantile to find the border of low prices
print('The lower quantile is {:.2f}'.format(
    ecommerce_data_orders['unit_price'].quantile(0.25)
))

In [ ]:
#calculating 0.75 quantile to find the border of high prices
print('The upper quantile is {:.2f}'.format(
    ecommerce_data_orders['unit_price'].quantile(0.75)
))

These values are too small to use them for making useful categories so we will try to work with histograms to do it visually.

On the previous histogram we saw that only few products cost more than 100 so we will build a new histogram without these outliers:

In [ ]:
#building a histgram with values less than 100
fig=px.histogram(ecommerce_data_orders[ecommerce_data_orders['unit_price'] < 100], x='unit_price', title='Distribution of prices')
fig.update_layout(xaxis_title='Price', yaxis_title='Count')
fig.show()

Here we see that between 20 and 100 are not many values so this wiil be our second border:

In [ ]:
#building a histogram with values less than 20
fig=px.histogram(ecommerce_data_orders[ecommerce_data_orders['unit_price'] < 20], x='unit_price', title='Distribution of prices')
fig.update_layout(xaxis_title='Price', yaxis_title='Count')
fig.show()

Finally we see that price of most of the products is between 0-5 so this is our next border. Also we can notice here that the most common range of prices is 1.63-1.67 and after 2.12 there is quite significant fall, so we will use this value as our last border.

Final product categories by price: 'very expensive' - 650-20, 'expensive'- 20-5, 'cheap' - 5-2.13, 'very cheap' - 2.12-0.04. 

We build a function to assign appropriate categories to the products by adding of a new column to the dataset:

In [ ]:
#building a function for splitting products by categories
def price_category(price):
    if price <= 2.12:
        return 'very cheap'
    if 2 <= price <= 5:
        return 'cheap'
    if 5 <= price <= 20:
        return 'expensive'
    return 'very expensive'

In [ ]:
#applying a function and creating new column in df
ecommerce_data_orders['price_category'] = ecommerce_data_orders['unit_price'].apply(price_category)
ecommerce_data_orders.sample()

Now we are going to define categories of products by their names. At first we will clean descriptions and make them more universal for easier searching of keywords: <a class="anchor" id="names"></a>

In [ ]:
##Creating a list of stop words 
stop_words = set(stopwords.words("english"))

In [ ]:
result=[]

for i in range(0, 522504):
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', ecommerce_data_orders['description'][i])
    
    text= ecommerce_data_orders['description'][i]
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    #Convert to list from string
    text = text.split()
    
    #Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    result.append(text)
    
ecommerce_data_orders['description_cleaned'] = result


In [ ]:
#defining common words in clean description to define popular keywords
count_words = pd.Series(' '.join(ecommerce_data_orders['description_cleaned'].unique()).split()).value_counts()[240:280]
#count_words

In [ ]:
#checking names of products with particular keyword before adding to the function
ecommerce_data_orders[ecommerce_data_orders['description_cleaned'].str.contains('crayon') == True]['description'].unique()


In [ ]:
#building a function for categorizing products
def categorize(keyword):
    if ('food cover' in keyword) or ('bowl' in keyword) or \
    ('bottle' in keyword) or ('mug' in keyword) or ('cup' in keyword) or \
    ('jar' in keyword) or ('cutlery' in keyword) or ('tray' in keyword) or \
    ('plate' in keyword) or ('dish' in keyword) or ('apron' in keyword) or \
    ('towel' in keyword) or ('cake case' in keyword) or ('cake tin' in keyword) or \
    ('mould' in keyword) or ('tea' in keyword) or ('baking' in keyword) or \
    ('lunch box' in keyword) or ('spoon' in keyword) or ('recipe box' in keyword) or \
    ('cutter' in keyword) or ('saucer' in keyword) or ('kitchen scale' in keyword) or \
    ('snack box' in keyword) or ('jam' in keyword) or ('spice tin' in keyword) or \
    ('chopping board' in keyword) or ('oven glove' in keyword):
        category= 'kitchen'
    elif ('necklace' in keyword) or ('bracelet' in keyword) or \
    ('earring' in keyword) or ('hair' in keyword) or ('key ring' in keyword) or \
    ('phone charm' in keyword) or ('bag charm' in keyword) or ('jewel' in keyword) or \
    ('brooch' in keyword) or ('trinket box' in keyword):
        category= 'accessories'
    elif ('drawer knob' in keyword) or ('metal sign' in keyword) or \
    ('doormat' in keyword) or ('doorstop' in keyword) or ('hook' in keyword) or \
    ('hanger' in keyword) or ('door sign' in keyword) or ('coat rack' in keyword) :
        category= 'hallway'
    elif ('paper lantern' in keyword) or ('paper bell' in keyword) or \
    ('paper ball' in keyword) or ('paper chain' in keyword) or ('garland' in keyword) or \
    ('bunting' in keyword) or ('wrap' in keyword) or \
    ('ribbon' in keyword) or ('christmas' in keyword) or \
    ('easter' in keyword) or ('napkin' in keyword) or ('tissue' in keyword) or \
    ('doily' in keyword) or ('cake stand' in keyword) or ('cakestand' in keyword) or \
    ('paper plates' in keyword) or ('balloon' in keyword) or ('party' in keyword) or ('bauble' in keyword) or \
    ('nesting' in keyword):
        category='party'
    elif ('notebook' in keyword) or ('sticker' in keyword) or \
    ('pencil' in keyword) or ('birthday card' in keyword) or ('greeting card' in keyword) or \
    ('set 10 card' in keyword) or ('passport cover' in keyword) or ('sketchbook' in keyword) or \
    ('memo board' in keyword):
        category='cabinet'
    elif ('light' in keyword) or ('candle' in keyword) or \
    ('decoration' in keyword) or ('holder' in keyword) or \
    ('clock' in keyword) or ('cushion cover' in keyword) or \
    ('cushion' in keyword) or ('block' in keyword) or \
    ('frame' in keyword) or ('magnet' in keyword) or \
    ('incense' in keyword) or ('wall art' in keyword) or ('ornament' in keyword) or \
    ('heart wicker' in keyword) or ('vintage billboard' in keyword) :
        category='home decoration'
    elif ('bag' in keyword) or ('purse' in keyword) or \
    ('wallet' in keyword) or ('hamper' in keyword) or \
    ('card holder' in keyword) or ('shopper' in keyword) :
        category='bags'
    elif 'bathroom' in keyword:
        category='bathroom'
    elif ('garden' in keyword) or ('umbrella' in keyword) or \
    ('parasol' in keyword):
        category='garden'
    elif ('toy' in keyword) or ('playing card' in keyword) or \
    ('slate' in keyword) or ('snap card' in keyword) or ('harmonica' in keyword) or \
    ('mouse' in keyword) or ('nancy' in keyword) or ('domino' in keyword) or \
    ('sewing kit' in keyword) or ('dollcraft' in keyword) or ('foxglove' in keyword) or \
    ('feltcraft doll' in keyword) or ('feltcraft princess' in keyword):
        category='toys and hobbies'
    else:
        category='other'
        
    return category
        

In [ ]:
#applying a function
ecommerce_data_orders['category'] = ecommerce_data_orders['description_cleaned'].apply(categorize)
ecommerce_data_orders['category'].value_counts()


In [ ]:
ecommerce_data_orders.groupby('category')['description_cleaned'].nunique()


In [ ]:
ecommerce_data_orders[ecommerce_data_orders['category'] == 'other']['description_cleaned'].value_counts().head(30)

We categorized all products on 10 categories and products with less common keywords make a category 'other'. 

Now we are going to analyze number of orders per customer - if there are many customers that buy frequently? What amount of orders is common and what is big? On which categories could we divide customers by number of orders they made? <a class="anchor" id="loyal"></a>

We start with calculating of number of orders per customer, then analyze its distribution:

In [ ]:
#calculating the number of orders per customer
num_orders_per_customer=ecommerce_data_orders.groupby('customer_id')['invoice_no'].nunique().reset_index()
num_orders_per_customer.drop(num_orders_per_customer[num_orders_per_customer['customer_id'] == 'unknown'].index, inplace=True)
num_orders_per_customer = num_orders_per_customer.rename(columns={'invoice_no': 'orders_num'})
num_orders_per_customer.sort_values(by='orders_num',ascending=False).head()


In [ ]:
#getting statistical data 
num_orders_per_customer.describe()

In [ ]:
#calculating 95th quantile
print('95%: {:.1f}'.format(num_orders_per_customer['orders_num'].quantile(0.95)))

In [ ]:
#building a histogram of distribution of number of orders per customer
fig=px.histogram(num_orders_per_customer,
                 x='orders_num', title='Distribution of orders per customer')
fig.update_layout(xaxis_title='Number of orders', yaxis_title='Count')
fig.show()

It seems that the most common number of orders is less than 28 and after there are significant outliers. We recognize them as anomaly amount of orders and cut them to build more informative histogram to see differences between smaller numbers. 

In [ ]:
#building a histogram without significant outliers
fig=px.histogram(num_orders_per_customer[num_orders_per_customer['orders_num'] < 29],
                 x='orders_num', title='Distribution of orders per customer')
fig.update_layout(xaxis_title='Number of orders', yaxis_title='Count')
fig.show()

The most common number of orders customers made is 1 - 1505, also many customers ordered 2 times - 831. 3-4 orders are also quite popular cases and more than 5 are less, but still there is significant number of customers that ordered up to 15 times.

In [ ]:
#calculating the number of casual customers that made only 1 order
print('Number of casual customers: {}'.format(
    num_orders_per_customer[num_orders_per_customer['orders_num'] ==1]['customer_id'].count()
))
print('Percentage of casual customers: {:.2f}%'.format(
    num_orders_per_customer[num_orders_per_customer['orders_num'] ==1]['customer_id'].count() / 
    ecommerce_data_orders['customer_id'].nunique() * 100
))


Thus, we found out that there are quite many casual customers (made only 1 order) - 1505 - 34.72% of all customers, but there are much more loyal customers that made 2 and more orders.

Now we will categorize customers by average check. We will calculate total price of purchased product, total size of each order, average check per customer and define what average sum of order is most common, how many customers make small and big orders. We start with calculating of total price of purchased product: <a class="anchor" id="avg"></a>

In [ ]:
#calculating the total price of each product
ecommerce_data_orders['total_price'] = ecommerce_data_orders['unit_price'] * ecommerce_data_orders['quantity']
ecommerce_data_orders = ecommerce_data_orders.round({'total_price': 2})
ecommerce_data_orders.head()


Now we can calculate total size of order per customer. We'll also remove 'unknown' customers on this step because these values will corrupt our further analysis of customers:

In [ ]:
#calculating total sum of the order per customer
total_order=ecommerce_data_orders.groupby(['customer_id', 'invoice_no'])['total_price'].sum().reset_index()
total_order.drop(total_order[total_order['customer_id'] == 'unknown'].index, inplace=True)
total_order = total_order.rename(columns={'total_price': 'total_sum'})
total_order = total_order.sort_values(by='total_sum', ascending=False)
total_order.head(10)

In [ ]:
#calculating the average check per customer
avg_check=total_order.groupby('customer_id')['total_sum'].mean().reset_index()
avg_check = avg_check.rename(columns={'total_sum': 'avg_check'})
avg_check = avg_check.sort_values(by='avg_check', ascending=False)
avg_check.head()

In [ ]:
avg_check['avg_check'].describe()

In [ ]:
#building a histogram of distribution of average checks
fig=px.histogram(avg_check, x='avg_check', title='Distribution of average checks')
fig.update_layout(xaxis_title='Average check', yaxis_title='Count')
fig.show()

The majority of the values are under 5000 and many close to 0, there are some significant outliers that we will straight away classificate as big orders and cut them to build more informative histogram:

In [ ]:
#building a histogram with values less than 5000
fig=px.histogram(avg_check[avg_check['avg_check'] < 5000], x='avg_check', title='Distribution of average checks')
fig.update_layout(xaxis_title='Average check', yaxis_title='Count')
fig.show()

This histogram is more convenient to analysis: we see here that quite many customers spent in average up to 350 and also significant amount spent 350-800 that can form two groups of customers. There is less significant amount of buyers that spent 800-1000 and after there are outliers so we will group them together to one more category.

In the end we have 3 categories of customers by average check: 1 - 3.75-350, 2 - 350-800, 3 - 800-85000.(This order because of number of customers in categories)

In [ ]:
#defining a function to split customers by the avg check
def avg_check_category(check):
    if check >= 800:
        return '3'
    if 350 <= check <= 800:
        return '2'
    else:
        return '1'

avg_check['customer_category'] = avg_check['avg_check'].apply(avg_check_category)
avg_check.sample()


In [ ]:
#calculating number of customers in each category
print('Number of customers in each category: ')
print('{}'.format(avg_check['customer_category'].value_counts()))

The first category - customers with the small average check - is the biggest, there are twice more customers as in the second and third is very tiny. So we can conclude that more common to make small orders, but also important to notice these huge average checks of the third group. 

We can also check if there is a tendency to make big orders among customers that make many orders(loyal customers):

In [ ]:
#merge num_orders with avg_check
customers= avg_check.merge(num_orders_per_customer, left_on='customer_id', right_on='customer_id')
customers.sample(10)

Good way to define relation between number of made orders and size of average check is building a scatterplot:

In [ ]:
#building a scatterplot
fig=px.scatter(customers, x='orders_num', y='avg_check', 
               color='customer_category', title='Relation between number of orders and average check',
               labels={
                   'orders_num' : 'Number of orders',
                   'avg_check' : 'Average check',
                   'customer_category' : 'Category'
               }             
              )
fig.show()

There are no such relation that customers that made many orders made big orders. There are also cases of big amount of orders and small average check (1 category) and significantly higher avg check and not many orders (3). There are also some outliers from all 3 categories that only confirm that there is no such relation. 

We will calculate a correlation between these columns to make sure that our conclusion is right:

In [ ]:
#calculating a correlation between num of orders and avg check
print('Correlation between average checks and number of orders is {}.'.format(
    customers['avg_check'].corr(customers['orders_num']).round(3)
))

There is no relation between number of made orders and size of average check.

## Conclusion

In this section, we categorized products and customers. We defined 4 categories of products by price using histograms:  'very cheap' - 0.04-2.12, 'cheap' - 2.13 - 5,  'expensive'- 5-20,  ‘very expensive' - 20-650. After that we defined categories of products by keywords: cleaned descriptions from digits and ‘stop words’, defined the most popular keywords, and wrote a function to split products by categories - ‘accessories’, ‘bags’, ‘bathroom’, ‘cabinet’, ‘garden’, ‘hallway’, ‘home decoration’, ‘kitchen’, ‘party’, ‘toys and hobbies’ and ‘other’. We also analyzed the number of orders per customer and defined that the most common number is 1-almost 35% of all customers are casual,but there are many regular customers that made 2 and more orders. 

We also divided customers into 3 categories by the average check - 1- 3.75-350, 2- 350-800, and 3 - 800-85000. The first category is the biggest, there are twice more customers as in the second and third is very tiny. So we can conclude that more common to make small orders, but also important to notice these huge average checks of the third group. We checked if there is a tendency among customers who made many orders to make big orders and there is no such.

## Analysis of seasonality of demand <a class="anchor" id="season"></a>

In this section we will check seasonal changes of demand (number of orders): <a class="anchor" id="season_gen"></a>

In [ ]:
#calculating the number of orders per day
season_demand=ecommerce_data_orders.groupby('invoice_date')['invoice_no'].count().reset_index()
season_demand = season_demand.rename(columns={'invoice_no': 'orders_num'})
season_demand.head()

In [ ]:
#building a lineplot for demand by season
fig = px.line(season_demand, x='invoice_date', y='orders_num', title = 'Demand by season')
fig.update_layout(xaxis_title='Date',yaxis_title= 'Orders')
fig.show()

Demand trends are stable throughout the period from January until November with two notable peaks - 16/04 and 28/08, it starts to increase at the beginning of November and reach the maximum value in 3/12(4/12 and 15/12 in 2018 are also busy days). It makes sense to assume that these peaks are related to the holidays - Easter and Christmas.

Now we'll check seasonality of demand by category of product: <a class="anchor" id="season_cat"></a>

In [ ]:
#extracting month from invoice_date
ecommerce_data_orders['month']=ecommerce_data_orders['invoice_date'].dt.month

In [ ]:
#grouping by month and category
season_demand_category=ecommerce_data_orders.groupby(['month', 'category'])['invoice_no'].count().reset_index()
season_demand_category = season_demand_category.rename(columns={'invoice_no': 'orders_num'})
season_demand_category.head()

In [ ]:
#building a histogram of demand by season
fig = px.bar(season_demand_category, x='month', y='orders_num', color= 'category', title = 'Demand by season')
fig.update_layout(xaxis_title='Date',yaxis_title= 'Orders')
fig.show()

November is the most active month, we see significant increase of orders of products almost of all categories - 'party', 'other' and 'kitchen' and 'home decoration' at most. October and December are also quite active months with some differences in popularity of categories('home decoration' and 'party' became less popular in December). Throughout the rest of the year trends of demand by category are about the same.

In [ ]:
#defining dfs by season
demand_winter = ecommerce_data_orders[(ecommerce_data_orders['invoice_date'] >= '2018-12-01') & (ecommerce_data_orders['invoice_date'] < '2019-03-01')]
demand_summer = ecommerce_data_orders[(ecommerce_data_orders['invoice_date'] >= '2019-06-01') & (ecommerce_data_orders['invoice_date'] < '2019-09-01')]
demand_spring = ecommerce_data_orders[(ecommerce_data_orders['invoice_date'] >= '2019-03-01') & (ecommerce_data_orders['invoice_date'] < '2019-06-01')]
demand_autumn = ecommerce_data_orders[(ecommerce_data_orders['invoice_date'] >= '2019-09-01') & (ecommerce_data_orders['invoice_date'] < '2019-12-01')]


In [ ]:
#defining the most popular products in winter
demand_winter['description_cleaned'].value_counts().head(30)

In [ ]:
#defining the most popular products in summer
demand_summer['description_cleaned'].value_counts().head(30)

In [ ]:
#defining the most popular products in spring
demand_spring['description_cleaned'].value_counts().head(30)

In [ ]:
#defining the most popular products in autumn
demand_autumn['description_cleaned'].value_counts().head(30)

Popular products definitely differ from season to season. We can notice the pattern of huge popularity of jumbo and lunch bags and buntings in summer, in spring bags are also quite popular, in autumn customers buy things that relate to christmas, in winter there is no clear pattern but there are many orders of things for home decorating and kitchen.

## Conclusion

At this stage, we analyzed the seasonality of demand. We found out that demand trends are stable almost all year with a peak in April and increasing in November that seems to relate to the holidays -  Easter and Christmas. Analysis of seasonality changes of the popularity of certain categories of products shows that throughout the year it’s quite stable with increasing in October-November, but for categories it’s proportional. We also checked the popularity of certain products by season and found few patterns for seasons- in summer jambo and lunch bags are at the top of popular goods, also buntings, in spring such bags are popular too, in autumn customers already buy things related to Christmas and in winter more home decorations and kitchen stuff. 

## Analysis of carts <a class="anchor" id="carts"></a>

Defining products that often bought together. We are going to use the Apriori algorithm, firstly prepared the data for it: <a class="anchor" id="together"></a>

In [ ]:
#creating a basket 
basket=(ecommerce_data_orders
        .groupby(['invoice_no','description_cleaned'])['quantity']
        .sum().unstack().reset_index().fillna(0)
        .set_index('invoice_no')
       )


In [ ]:
basket.head()

In [ ]:
#converting values to 0 and 1
def convert(num):
    if num <=0:
        return 0
    else:
        return 1

basket_prepared=basket.applymap(convert)
#basket_prepared

In [ ]:
# Building the model
frq_items = apriori(basket_prepared, min_support = 0.02, use_colnames = True)
 
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])


In [ ]:
pd.set_option('max_colwidth', 1)
rules=rules[['antecedents','consequents', 'support', 'confidence', 'lift']].round(3)
#rules.head()


In [ ]:
rules.head(50)

Detected combinations of goods that are often bought together are mostly two (or more) kinds of the same products that differ by color or design. There are very few examples at the top of the list of combinations of different products, such as cake stand tier and teacup saucer.

Now we are going to define what products are often bought in big amounts. We will check products of what price category are more popular for buying in several copies. <a class="anchor" id="bigamount"></a>

In [ ]:
#getting statistics about 'very cheap' category
ecommerce_data_orders[ecommerce_data_orders['price_category'] == 'very cheap'][['quantity','total_price']].describe()


In [ ]:
#calculating 95th quantile because max value is anomalous
print('95%: {}'.format(
    ecommerce_data_orders[ecommerce_data_orders['price_category'] == 'very cheap']['quantity'].quantile(0.95)
))


In [ ]:
#plotting a histogram of distribution of quantity of very cheap products
fig=px.histogram(ecommerce_data_orders[(ecommerce_data_orders['price_category'] == 'very cheap') & (ecommerce_data_orders['quantity'] < 100)], x='quantity', title='Distribution of quantity of very cheap products')
fig.update_layout(xaxis_title='Quantity', yaxis_title='Count')
fig.show()

There are a lot of sold very cheap products - 280011, many of them were bought in a single copy, but more in few. The most common quantity is 12. There are also anomalous outliers - after 95th quantile that equal to 48 units there are much more up to 80995. 

In [ ]:
#checking statisctics for cheap products
ecommerce_data_orders[ecommerce_data_orders['price_category'] == 'cheap']['quantity'].describe()


In [ ]:
#plotting a histogram of distribution of quantity of cheap products
fig=px.histogram(ecommerce_data_orders[(ecommerce_data_orders['price_category'] == 'cheap') & (ecommerce_data_orders['quantity'] < 100)], 
                 x='quantity', 
                 title='Distribution of quantity of cheap products')
fig.update_layout(xaxis_title='Quantity', yaxis_title='Count')
fig.show()

There are significantly less sold products in this category - by ~130000. Outliers also less significant- max quantity is 1930. The most common quantity is 1 and the next value after it less than twice. Significant number of goods were sold in 6 copies, rest quantities are much less common.

In [ ]:
#checking statistics of axpensive goods
ecommerce_data_orders[ecommerce_data_orders['price_category'] == 'expensive']['quantity'].describe()


In [ ]:
#plotting a histogram of distribution of quantity of expensive products
fig=px.histogram(ecommerce_data_orders[(ecommerce_data_orders['price_category'] == 'expensive') & (ecommerce_data_orders['quantity'] < 100)],
                 x='quantity', 
                 title='Distribution of quantity of expensive products')
fig.update_layout(xaxis_title='Quantity', yaxis_title='Count')
fig.show()

There are two times less sold products in this category than in previous. The most common quantity is 1, the 2d is 2 and these more than half of all sold products in this category. Maximum value also smaller - 1412. This is less common case to buy many units of products of this category

In [ ]:
#getting statistics on the 'very expensive' category
ecommerce_data_orders[ecommerce_data_orders['price_category'] == 'very expensive']['quantity'].describe()

In [ ]:
#plotting a histogram of distribution of quantity of very expensive products
fig=px.histogram(ecommerce_data_orders[ecommerce_data_orders['price_category'] == 'very expensive'],
                 x='quantity', 
                 title='Distribution of quantity of very expensive products')
fig.update_layout(xaxis_title='Quantity', yaxis_title='Count')
fig.show()

In [ ]:
ecommerce_data_orders.to_csv('ecommerce_data_orders.csv', index=False)

This category is the smallest, only 2581 units were sold, the absolute majority - 1950 in only one copy, much less in 2. Maximum value is 125. There is no such tendency to buy very expensive products in big amount.

## Conclusion

We checked what combinations of goods are popular among customers using an Apriori algorithm and found mostly orders of similar products that differ by color or design. We also analyzed what category of products is most popular to orders of many units together and found that very cheap products are absolute leaders here - the most common quantity is 12 units per order. There are much fewer sold products of the rest categories and the most common quantity for them is 1. 

## Analysis of canceled orders <a class="anchor" id="canceled"></a>

In this section we'll calculate a share of canceled orders, average sum, and define what products often canceled.

In [ ]:
ecommerce_data_canceled.head()

In [ ]:
#calculating the number of canceled orders
num_canceled = ecommerce_data_canceled['invoice_no'].nunique()
print('Number of canceled orders: {}'.format(num_canceled))

In [ ]:
#calculating the total number of orders
num_orders= ecommerce_data['invoice_no'].nunique()
print('Total number of orders: {}'.format(num_orders))

In [ ]:
#calculating a share of canceled orders
share = num_canceled / num_orders * 100
print('Share of canceled orders: {:.2f}%'.format(share))

In [ ]:
#calculating the total price of each product
ecommerce_data_canceled['total_price'] = ecommerce_data_canceled['unit_price'] * (-1) * ecommerce_data_canceled['quantity'] 
ecommerce_data_canceled = ecommerce_data_canceled.round({'total_price': 2})
ecommerce_data_canceled.head()


In [ ]:
#calculating total sum of the order per customer
total_canceled_order=ecommerce_data_canceled.groupby(['customer_id', 'invoice_no'])['total_price'].sum().reset_index()
total_canceled_order = total_canceled_order.rename(columns={'total_price': 'total_sum'})
total_canceled_order = total_canceled_order.sort_values(by='total_sum', ascending=False)
total_canceled_order.head()


In [ ]:
total_canceled_order['total_sum'].describe()

There are few significant outliers, customer 16446 we already known from searching of total sum of order per customer and this is the same number so he doesn't enter to our analysis. The same thing with the customer 12346. Customer 15749 we saw with one of the biggest average check so he is also out of analysis.

In [ ]:
#building a histogram of distribution of canceled orders by total sum
fig=px.histogram(total_canceled_order[total_canceled_order['total_sum'] < 9000], x='total_sum', 
                 title='Distribution of canceled orders by total sum')
fig.update_layout(xaxis_title='Total sum', yaxis_title='Count')
fig.show()


We see that there are few canceled oreders on very big sum in range 100-9000, but the absolute majority (2069) are orders on sum less than 20. Calculating the average sum of canceled order here doesn't make sense, but we can calculate mode to define the most common sum:

In [ ]:
print('Mode of total sum of canceled orders: {}'.format(
    total_canceled_order['total_sum'].mode()[0]
))


Now we want to find products that were frequently canceled.

In [ ]:
#merge total_sum to df with canceled orders
ecommerce_data_canceled= ecommerce_data_canceled.merge(total_canceled_order, on=['customer_id', 'invoice_no'])
ecommerce_data_canceled.head(2)

In [ ]:
#what products often canceled?
ecommerce_data_canceled['description'].value_counts().head(20)

'REGENCY CAKESTAND 3 TIER' was in 180 canceled orders that quite big number, but there is big different between it to the 2d place - 87 cancelations of 'JAM MAKING SET WITH JARS'. These goods were in canceled orders, so we can't check if there are any problem with them specifically because there are many possible reasons to cancel the whole order. But we can say that there are relatively not too much cancelations, but reasons should be checked to reduce them more. Probably we have a problem on a page with a confirmation of order?

## Conclusion

The share of canceled orders is 14.35% which is quite many. There is a wide range of total sums of canceled orders with few significant outliers which seems like a mistake, probably these customers didn’t want to do an order at all. The majority of the canceled orders are within 20, the most common sum is 4.95(we didn’t calculate an average sum because of the too big spread of values). We checked what products are often canceled and found that ‘REGENCY CAKESTAND 3 TIER’ was canceled the most times - 180, and on the 2d place 'JAM MAKING SET WITH JARS', there is significant difference between these number and after that they continue to decrease quickly. We can't say why these products were in canceled orders, but possible reasons of cancelations of whole orders should be checked,probably it's relate to any technical issues.

## Testing statistical hypotheses <a class="anchor" id="hypotheses"></a>

Finally we want to test two statistical hypotheses: 
1. If there is a statistically significant difference between revenue from small products that were bought in big amounts and big expensive products that bought one at a time?
2. If there is a statistically significant difference between revenue in summer and winter?

#### First hypothesis <a class="anchor" id="first"></a>
First of all we are going to define samples for testing. We will use our previously defined categories of products 'very cheap' and 'very expensive' that perfectly fit our conditions. Customers usually buy multiple units of products from category 'very cheap' and only one at time from category 'very expensive'.

In [ ]:
#group the data by category and price to find unique products per category and price
grouped_category=ecommerce_data_orders.groupby(['price_category', 'total_price'])['description_cleaned'].unique().reset_index()
grouped_category.head(1)

In [ ]:
#taking a samples from grouped data
very_cheap=grouped_category[grouped_category['price_category'] == 'very cheap']['total_price']
print('Size of sample with cheap products: {}'.format(very_cheap.shape[0]))
very_expensive=grouped_category[grouped_category['price_category'] == 'very expensive']['total_price']
print('Size of sample with expensive products: {}'.format(very_expensive.shape[0]))

Before the testing of our hypothesis we need to check samples for normality of distribution. We will do it by using Q-Q plot:

In [ ]:
#plotting the quantile-quantile probability plot
z = (very_cheap-np.mean(very_cheap))/np.std(very_cheap)

stats.probplot(z, dist="norm", plot=plt)
plt.title("Probability Plot of 'very_cheap' sample")
plt.show()

In [ ]:
#defining an outlier that can corrupt the result of the test
print("The maximum value of 'very_cheap' sample: {}".format(very_cheap.max()))
print("Index of the maximum value: {}".format(very_cheap.idxmax()))


In [ ]:
#removing the outlier
very_cheap=very_cheap.drop(index=[4458])

In [ ]:
print("The maximum value of 'very_cheap' sample: {}".format(very_cheap.max()))
print("Index of the maximum value: {}".format(very_cheap.idxmax()))


In [ ]:
#removing the outlier
very_cheap=very_cheap.drop(index=[4457])

In [ ]:
#plotting the quantile-quantile probability plot
z = (very_cheap-np.mean(very_cheap))/np.std(very_cheap)

stats.probplot(z, dist="norm", plot=plt)
plt.title("Probability Plot of 'very_cheap' sample")
plt.show()

In [ ]:
#plotting the quantile-quantile probability plot
z = (very_expensive-np.mean(very_expensive))/np.std(very_expensive)

stats.probplot(z, dist="norm", plot=plt)
plt.title("Probability Plot of 'very_expensive' sample")
plt.show()

This sample is not follow a normal distribution.

In [ ]:
#defining an outlier that can corrupt the result of the test
print("The maximum value of 'very_expensive' sample: {}".format(very_expensive.max()))
print("Index of the maximum value: {}".format(very_expensive.idxmax()))


In [ ]:
#removing the outlier
very_expensive=very_expensive.drop(index=[4620])

In [ ]:
#plotting the quantile-quantile probability plot
z = (very_expensive-np.mean(very_expensive))/np.std(very_expensive)

stats.probplot(z, dist="norm", plot=plt)
plt.title("Probability Plot of 'very_expensive' sample")
plt.show()

The sample is not follow the normal distribution.

Because of data is not distributed normally we will use 'Mann-Whitney U Test'.

H0: There are statistical insignificant differences between the average revenue from very cheap and very expensive products.

Ha: There are statistical significant differences between the average revenue from very cheap and very expensive products.

Level of statistical significance we will use is 0.05.

In [ ]:
alpha = 0.05
results = mannwhitneyu(very_cheap, very_expensive)
print('p-value:', results.pvalue)
if results.pvalue < alpha:
    print('We reject the null hypothesis')
else:
    print("We can't reject the null hypothesis")

Result of the test is rejecting of the null hypothesis - average revenues from cheap and expensive products are significantly different.

#### Second Hypothesis <a class="anchor" id="second"></a>

In [ ]:
#grouping winter data to get total sum of order
total_order_winter=demand_winter.groupby(['invoice_no'])['total_price'].sum().reset_index()
total_order_winter = total_order_winter.rename(columns={'total_price': 'total_order'})
total_order_winter.head()

In [ ]:
#grouping summer data to get total sum of order
total_order_summer=demand_summer.groupby(['invoice_no'])['total_price'].sum().reset_index()
total_order_summer = total_order_summer.rename(columns={'total_price': 'total_order'})
total_order_summer.head()

In [ ]:
#defining sizes of samples
print('Size of sample with winter orders: {}'.format(total_order_winter.shape[0]))
print('Size of sample with summer orders: {}'.format(total_order_summer.shape[0]))


In [ ]:
#plotting a histogram for the summer sample to define normality of distribution
plt.hist(total_order_summer['total_order'], bins='auto')
plt.show()

In [ ]:
#defining an outlier that can corrupt the result of the test
print("The maximum value of summer sample: {}".format(total_order_summer['total_order'].max()))
print("Index of the maximum value: {}".format(total_order_summer['total_order'].idxmax()))


In [ ]:
#removing the outlier
total_order_summer=total_order_summer.drop(index=[471])

In [ ]:
#plotting a new histogram
plt.hist(total_order_summer['total_order'], bins='auto')
plt.show()

The data is not distributed normally.

In [ ]:
#plotting a histogram of the winter sample to define normality of distribution
plt.hist(total_order_winter['total_order'], bins='auto')
plt.show()

In [ ]:
#defining an outlier and its index
print("The maximum value of winter sample: {}".format(total_order_winter['total_order'].max()))
print("Index of the maximum value: {}".format(total_order_winter['total_order'].idxmax()))


In [ ]:
#removing the outlier
total_order_winter=total_order_winter.drop(index=[1839])

In [ ]:
#plotting a new histogram
plt.hist(total_order_winter['total_order'], bins='auto')
plt.show()

The data is not distributed normally.

Because of data is not distributed normally we will use 'Mann-Whitney U Test'.

H0: There are statistical insignificant differences between the average revenue in summer and winter.

Ha: There are statistical significant differences between the average revenue in summer and winter.

Level of statistical significance we will use is 0.05.

In [ ]:
alpha = 0.05
results = mannwhitneyu(total_order_winter['total_order'], total_order_summer['total_order'])
print('p-value:', results.pvalue)
if results.pvalue < alpha:
    print('We reject the null hypothesis')
else:
    print("We can't reject the null hypothesis")

By the result of the test we can't reject the null hypothesis that means that average revenue in summer and winter is not differ significantly.

## Conclusion

We tested two statistical hypotheses and found out that differences between average revenues from cheap and expensive products are statistically significant, and between average revenues in winter and summer aren't.

## Overall Conclusion <a class="anchor" id="overall"></a>

* We got a dataset with 541909 entries that contain information about orders made in the store: order number and date, identifier and name of the product, purchased quantity, price per unit, customer id.
* Data was preprocessed: we changed column names and corrected datatypes, removed null values from column ‘description’, filled null values in column ‘customer_id’ with ‘unknown’, removed duplicates. Strange irrelevant values in column ‘stock_code’ were deleted. Dataset was split into two - canceled orders in the separate dataset.
* We analyzed prices and defined 4 categories of products by price: 'very cheap' - 0.04-2.12, 'cheap' - 2.13 - 5, 'expensive'- 5-20, ‘very expensive' - 20-650. We also extracted keywords from descriptions and formed 11 categories of products: ‘accessories’, ‘bags’, ‘bathroom’, ‘cabinet’, ‘garden’, ‘hallway’, ‘home decoration’, ‘kitchen’, ‘party’, ‘toys and hobbies’ and ‘other’. 
* We found out that the most common number of orders per customer is 1. The most customers are regular - 2829 and 1505 made order once.
* We categorized customers by the average check: 1- 3.75-350, 2- 350-800, and 3 - 800-85000. The first category is the biggest and we conclude that it’s more common to make small orders, but huge sums of the third group should be noticed. We checked if there is a tendency among customers who made many orders(loyal) to make big orders and there is no such.
* We analyzed the seasonality of demand and found out that there are peaks in April and November, most likely related to Easter and Christmas. The popularity of categories is stable throughout the year and proportional during the peaks. There are few patterns of the popularity of certain products by season, such as in summer jambo and lunch bags are at the top of popular goods, also buntings, in spring such bags are popular too, in autumn customers already buy things related to Christmas and in winter more home decorations and kitchen stuff. 
* We checked what combinations of goods are popular among customers and found mostly orders of similar products that differ by color or design. We also analyzed what category of products is most popular to orders of many units together and found that very cheap products are absolute leaders here - the most common quantity is 12 units per order. There are much fewer sold products of the rest categories and the most common quantity for them is 1.
* Analysis of canceled orders showed that the share of canceled orders is  14.35% which is quite many. With the information that we have, we can’t check what it caused by so it should be checked, probably we can reduce this number. 
* We also tested two statistical hypotheses and found out that differences between average revenues from cheap and expensive products are statistically significant, and between average revenues in winter and summer aren't.

Thus we can conclude that studied indicators are good: 
- we had 4334 customers this year and 2829(65%) made more than 1 order so they are loyal and possibly will return in the future, we need to attract them with special promotions, emails, exclusive offers for buying the same products, etc.
- we have a very perspective category of ‘very cheap’ products with prices 0.04-2.12, we sold 280011 such units, we can increase these sales by adding more similar goods and special offers for orders in big amount. Products of the category ‘very expensive’ should be checked, maybe the most expensive ones less suitable for our store or we oppositely should expand this category.
- we had significant peaks of demand in Easter and Christmas because we sell a lot of stuff for holidays. We can make a sale with special offers in these periods, we can add more goods related to parties and specific holidays to even increase these peaks. 
- the amount and sums of canceled orders is not crucial, but still, we can reduce it by checking the possible reasons and if it depends on us - fix it.